# Import Dataset and Library

In [ ]:
import pandas as pd
import gdown

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
# Helper Function
def import_from_gdrive(id_file, file_name, sheet_name=None):
    link = 'https://drive.google.com/uc?id=' + id_file
    output_file = '/content/' + file_name
    gdown.download(link, output_file, quiet=False)

    df = pd.read_excel(output_file)
    return df

In [ ]:
id_file = '1aPZBBtGwqKiPN9fJpY-LUTCz6mVDuux2'
df = import_from_gdrive(id_file,'final_dataset_converted_rupiah_outlier_handling.xlsx')

Downloading...
From: https://drive.google.com/uc?id=1aPZBBtGwqKiPN9fJpY-LUTCz6mVDuux2
To: /content/final_dataset_converted_rupiah_outlier_handling.xlsx
100%|██████████| 9.83M/9.83M [00:00<00:00, 18.5MB/s]


In [ ]:
fix_columns =['Total Household Income',
              'Total Household Income (Monthly)',
              'Household Head Sex',
              'Household Head Age',
              'Household Head Marital Status',
              # 'Household Head Highest Grade Completed',
              'Household Head Highest Grade Completed (Simplified)',
              'Region',
              'Type of Building/House',
              'House Floor Area',
              'Number of bedrooms',
              'Electricity',
              'Tenure Status',
              'Type of Household',
              'Total Number of Family members',
              'Total number of family members employed',
              # 'Number of Car, Jeep, Van',
              # 'Number of Motorcycle/Tricycle',
              # 'Number of Television',
              # 'Number of Airconditioner',
              # 'Number of Personal Computer',
              # 'Number of Cellular phone',
              # 'Number of Refrigerator/Freezer',
              # 'Number of Washing Machine',
              'Number of Kids',
              'Number of Vehicles',
              'Number of Communication Devices',
              'Number of Electronics',
              'Total Expenses',
              ]

df_selected = df[fix_columns]

# Linear Regression

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error

In [ ]:
# Memisahkan fitur dan target
X_lr = df_selected.drop(columns=['Total Expenses'])
y_lr = df_selected['Total Expenses']

In [ ]:
# Memilih fitur numerik dan kategorikal
numeric_features = X_lr.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_lr.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline preprocessing untuk fitur numerik menggunakan RobustScaler
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())  # Robust Scaling untuk mengatasi outlier
])

# Pipeline preprocessing untuk fitur kategorikal
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
])

# Gabungkan pipeline preprocessing untuk fitur numerik dan kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Membangun pipeline dengan pra-pemrosesan dan model regresi
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Menentukan skor evaluasi yang akan digunakan dalam cross-validation
scoring = {
    'mae': make_scorer(mean_absolute_error),
    'mse': make_scorer(mean_squared_error)
}


In [ ]:
# Melakukan k-fold cross-validation
cv_results = cross_validate(model_pipeline, X_lr, y_lr, cv=5, scoring=scoring, return_train_score=True)

# Mencetak hasil evaluasi
print(f"Mean Absolute Error (MAE) for each fold: {cv_results['test_mae']}")
print(f"Mean Squared Error (MSE) for each fold: {cv_results['test_mse']}")
print(f"Average Mean Absolute Error (MAE): {cv_results['test_mae'].mean()}")
print(f"Average Mean Squared Error (MSE): {cv_results['test_mse'].mean()}")

Mean Absolute Error (MAE) for each fold: [17520998.98790312 18108921.87033296 17027009.39741151 17342501.32806402
 19915335.92339411]
Mean Squared Error (MSE) for each fold: [5.61960639e+14 5.77800254e+14 5.23665782e+14 5.51931328e+14
 7.04523503e+14]
Average Mean Absolute Error (MAE): 17982953.501421146
Average Mean Squared Error (MSE): 583976301168644.2


In [ ]:
"""

Liner Regression with all column
Mean Absolute Error (MAE) for each fold: [610.95603463 615.62203511 615.95692623 596.74041253 706.81376232]
Mean Squared Error (MSE) for each fold: [1691312.55015805 2096810.43604021 2116739.3277746  2453121.44717996
 2900785.21903759]
Average Mean Absolute Error (MAE): 629.2178341652432
Average Mean Squared Error (MSE): 2,251,753.7960380805

Liner Regression with all column
Mean Absolute Error (MAE) for each fold: [17520998.98790312 18108921.87033296 17027009.39741151 17342501.32806402
 19915335.92339411]
Mean Squared Error (MSE) for each fold: [5.61960639e+14 5.77800254e+14 5.23665782e+14 5.51931328e+14
 7.04523503e+14]
Average Mean Absolute Error (MAE): 17,982,953.501421146
Average Mean Squared Error (MSE): 583,976,301,168,644.2

"""

'\n\nLiner Regression with all column\nMean Absolute Error (MAE) for each fold: [610.95603463 615.62203511 615.95692623 596.74041253 706.81376232]\nMean Squared Error (MSE) for each fold: [1691312.55015805 2096810.43604021 2116739.3277746  2453121.44717996\n 2900785.21903759]\nAverage Mean Absolute Error (MAE): 629.2178341652432\nAverage Mean Squared Error (MSE): 2,251,753.7960380805\n\n\n\n'

# Random Forest Regressor

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Memisahkan fitur dan target
X_rf = df_selected.drop(columns=['Total Expenses'])
y_rf = df_selected['Total Expenses']

In [ ]:
# Memilih fitur numerik dan kategorikal
numeric_features = X_rf.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_rf.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline preprocessing untuk fitur numerik menggunakan RobustScaler
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())  # Robust Scaling untuk mengatasi outlier
])

# Pipeline preprocessing untuk fitur kategorikal
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
])

# Gabungkan pipeline preprocessing untuk fitur numerik dan kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Membangun pipeline dengan pra-pemrosesan dan model random forest regressor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Menentukan skor evaluasi yang akan digunakan dalam cross-validation
scoring = {
    'mae': make_scorer(mean_absolute_error),
    'mse': make_scorer(mean_squared_error)
}

# Best parameters: {'regressor__max_depth': 20, 'regressor__min_samples_leaf': 4, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 300}
# Best MAE: -16108297.716134787



In [ ]:
# Melakukan k-fold cross-validation
cv_results = cross_validate(model_pipeline, X_rf, y_rf, cv=5, scoring=scoring, return_train_score=True)

# Mencetak hasil evaluasi
print(f"Mean Absolute Error (MAE) for each fold: {cv_results['test_mae']}")
print(f"Mean Squared Error (MSE) for each fold: {cv_results['test_mse']}")
print(f"Average Mean Absolute Error (MAE): {cv_results['test_mae'].mean()}")
print(f"Average Mean Squared Error (MSE): {cv_results['test_mse'].mean()}")

Mean Absolute Error (MAE) for each fold: [16559519.05802121 16861553.60338182 15646000.4955561  15274632.20688015
 17175295.35908713]
Mean Squared Error (MSE) for each fold: [5.03280323e+14 5.14801900e+14 4.59506880e+14 4.25311335e+14
 5.33355769e+14]
Average Mean Absolute Error (MAE): 16303400.144585285
Average Mean Squared Error (MSE): 487251241217625.7


In [ ]:
df['Total Expenses'].describe()

count    2.193000e+04
mean     1.123700e+08
std      3.908032e+07
min      6.450324e+07
25%      8.121726e+07
50%      1.018247e+08
75%      1.350025e+08
max      2.283958e+08
Name: Total Expenses, dtype: float64

In [ ]:


"""
Mean Absolute Error (MAE) for each fold: [16559519.05802121 16861553.60338182 15646000.4955561  15274632.20688015
 17175295.35908713]
Mean Squared Error (MSE) for each fold: [5.03280323e+14 5.14801900e+14 4.59506880e+14 4.25311335e+14
 5.33355769e+14]
Average Mean Absolute Error (MAE): 16,303,400.144585285
Average Mean Squared Error (MSE): 487,251,241,217,625.7

81,217,260.0

"""

In [50]:
# Pipeline model dengan preprocessor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Definisikan parameter grid
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid,
                           scoring='neg_mean_absolute_error', cv=5, n_jobs=-1, verbose=2)

# Fit GridSearchCV
grid_search.fit(X_rf, y_rf)

# Cetak parameter terbaik dan MAE terbaik
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best MAE: {grid_search.best_score_}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters: {'regressor__max_depth': 20, 'regressor__min_samples_leaf': 4, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 300}
Best MAE: -16108297.716134787
